# Experiment : Internal variable growth

## Init the Ray client

In [1]:
# Imports
from clases.red_rddas_model import RedRddasModel
import ray
import time
import pandas as pd
ray.shutdown()
# Ray Configurations
ray.shutdown()
runtime_env = {"working_dir": "/home/reynaldo/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
ray.init(address='ray://172.17.163.253:10001', runtime_env=runtime_env, log_to_driver=False, allow_multiple=True, ignore_reinit_error=True)
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False, num_cpus=12)
# ray.init(log_to_driver=False, num_cpus=12)
# ray.init(log_to_driver=False)

2022-07-18 20:30:25,567	INFO packaging.py:414 -- Creating a file package for local directory '/home/reynaldo/Documents/RESEARCH/SynEstRDDA'.
2022-07-18 20:30:25,613	WARNING packaging.py:301 -- File /home/reynaldo/Documents/RESEARCH/SynEstRDDA/.git/objects/pack/pack-83a73ad721eaae41fd5b05d906bd0582e67c27b6.pack is very large (11.31MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/reynaldo/Documents/RESEARCH/SynEstRDDA/.git/objects/pack/pack-83a73ad721eaae41fd5b05d906bd0582e67c27b6.pack']})`
2022-07-18 20:30:25,737	INFO packaging.py:258 -- Pushing file package 'gcs://_ray_pkg_4d571328255a9aee.zip' (25.89MiB) to Ray cluster...
2022-07-18 20:30:26,406	INFO packaging.py:267 -- Successfully pushed file package 'gcs://_ray_pkg_4d571328255a9aee.zip'.


ClientContext(dashboard_url='', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', protocol_version='2022-03-16', _num_clients=4, _context_to_restore=<ray.util.client._ClientContext object at 0x7f120479fc10>)

### Generate the Samples and recollected the indicators

In [2]:
#capture the time for all the experiment
v_begin_exp = time.time()
# Experiment for internal variable growth
n_experiments = 500
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_of_var_by_rdd_min = 5
    n_of_var_by_rdd_max = 10

    # Fixed Parameters
    n_of_rdds = 5
    n_of_signals_rdd = 2
    n_output_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_variables_rdd in range(n_of_var_by_rdd_min, n_of_var_by_rdd_max + 1):
        print("Number of Network:", v_n_network)
        print("Number of RDDs:", n_of_rdds)
        print("Number of var:", n_of_variables_rdd)
        print("-------------------------------")


        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        o_rdda_model = RedRddasModel(n_of_rdds, n_of_variables_rdd, n_of_signals_rdd, n_output_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        o_rdda_model.generate_rddas(type_network=type_network)

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_variables": n_of_variables_rdd,
                    "n_coupling_signals": n_of_signals_rdd,
                    "n_rdda_attractors": len(o_rdda_model.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(o_rdda_model.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(o_rdda_model.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
# Capture the time of the experiment
v_end_exp = time.time()
v_time_exp = v_end_exp - v_begin_exp
# print("Time of Experiment (in seg)", v_time_exp)
# print("Time of Experiment (in hours, minutes and seconds)", time.strftime("%H:%M:%S", time.gmtime(v_time_exp)))
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
Number of RDDs: 5
Number of var: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...


2022-07-18 20:30:31,368	ERROR services.py:1488 -- Failed to start the dashboard: Ray Client is not connected. Please connect by calling `ray.init`.
2022-07-18 20:30:31,370	ERROR services.py:1489 -- Ray Client is not connected. Please connect by calling `ray.init`.
Traceback (most recent call last):
  File "/home/reynaldo/.local/lib/python3.8/site-packages/ray/_private/services.py", line 1425, in start_dashboard
    dashboard_url = ray.experimental.internal_kv._internal_kv_get(
  File "/home/reynaldo/.local/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return getattr(ray, func.__name__)(*args, **kwargs)
  File "/home/reynaldo/.local/lib/python3.8/site-packages/ray/util/client/__init__.py", line 267, in __getattr__
    return self.get_context().__getattr__(name)
  File "/home/reynaldo/.local/lib/python3.8/site-packages/ray/util/client/__init__.py", line 169, in __getattr__
    raise Exception(
Exception: Ray Client is not connected. Please connec

AssertionError: 

### Resume of the experiment

In [ ]:
# Time of Experiment (in seg) 1898.2322750091553
# Time of Experiment (in hours, minutes and seconds) 00:31:38
print("RESUME OF THE EXPERIMENT")
print("--------------------------------------------------------------------")
print("Name of the Experiment:", "Experiment 2 - Internal Variable Growth")
print("Variable Parameters : Number os Variables")
print("Range of Number of variables:",n_of_var_by_rdd_min,"-", n_of_var_by_rdd_max)
print("Fixed parameters")
print("Number of rdds:",n_of_rdds)
print("Number of signals by RDD :", n_of_signals_rdd)
print("Number of output variables:", n_output_variables)
print("Number of function clauses:", n_clauses_function)
print("Network Type:", type_network)
print("--------------------------------------------------------------------")
print("Time of Experiment (in seg)", v_time_exp)
print("Time of Experiment (in hours, minutes and seconds)", time.strftime("%H:%M:%S", time.gmtime(v_time_exp)))
print("--------------------------------------------------------------------")
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

### Save the experiment data in csv, using pandas Dataframe

In [ ]:
path = "exp2_internal_variable_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)